In [37]:
import openai
import pandas as pd
import re

# Your API key here
openai.api_key = "sk-aCxLYPM7ksEW5hRC0p80hQUs0LGuw5SQGSFgL1URxcT3BlbkFJG7J7YTe0SluXJZtU1ZMEr_y2VmtsZHIbn1nBsapmsA" 


In [38]:
# reviews_general_selected  = pd.read_csv("C:/Users/Theresa/Downloads/reviews_general_selected.csv")
# reviews_additional_selected = pd.read_csv("C:/Users/Theresa/Downloads/reviews_additional_selected.csv")

In [39]:
# # merging the data
# reviews = pd.merge(reviews_general_selected, reviews_additional_selected, on='review_id', how='outer')

# # creating a subset
# reviews_subset = reviews.head(10)

In [40]:
subratings_data  = pd.read_csv("C:/Users/Theresa/Downloads/subratings_data.csv")

reviews_subset = subratings_data.head(300)

In [41]:
import pandas as pd
import re
import openai
from collections import defaultdict

def compute_metrics(true_ratings, predicted_ratings):
    """
    Compute accuracy and MAE for non-None predicted ratings.
    """
    valid_pairs = [(true, pred) for true, pred in zip(true_ratings, predicted_ratings) if pred is not None]
    if not valid_pairs:
        return 0, None  # Handle case where no valid predictions exist

    valid_true, valid_pred = zip(*valid_pairs)
    accuracy = sum(1 for true, pred in valid_pairs if true == pred) / len(valid_pairs) * 100
    mae = sum(abs(true - pred) for true, pred in valid_pairs) / len(valid_pairs)
    return accuracy, mae

def extract_topics(sentence):
    prompt = (
        "For the following sentence, identify all applicable categories: "
        "'food', 'service', 'atmosphere', 'price'. If no category applies, respond 'none'. "
        "Separate multiple categories with commas.\n\n"
        "Sentence: {sentence}\n\n"
        "Categories:"
    )

    try: 
        # Call the OpenAI API with the custom prompt
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[ 
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt.format(sentence=sentence)}
            ],
            max_tokens=150
        )
        
        generated_text = response.choices[0].message['content'].strip()
        return generated_text

    except Exception as e:
        print(f"Error in extracting topics: {e}")
        return None


def analyze_sentiment(sentences):
    sentiment_prompt = (
        "Rate the sentiment of the following sentences on a scale of 1 to 5, "
        "where 1 is 'very bad' and 5 is 'very good'. Only provide the rating as a single number.\n\n"
        "Sentences: {sentences}\n\n"
        "Rating:"
    )

    try:
        # Combine all sentences for sentiment analysis
        combined_sentences = " ".join(sentences)
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": sentiment_prompt.format(sentences=combined_sentences)}
            ],
            max_tokens=10
        )
        
        rating = response.choices[0].message['content'].strip()
        return int(rating)

    except Exception as e:
        print(f"Error in analyzing sentiment: {e}")
        return None


def process_reviews(reviews_subset):
    # Create a list to store results
    results = []

    for index, row in reviews_subset.iterrows():
        full_review = row['review_text']  # Full review text
        
        # Check if the review is missing or invalid
        if pd.isna(full_review) or not isinstance(full_review, str):
            print(f"Skipping review {index} due to invalid review text.")
            continue
        
        sentences = re.split(r'(?<=[.!?])\s+', full_review.strip())  # Split into sentences
        
        # Dictionary to store sentences by topic
        topic_groups = defaultdict(list)
        
        for sentence in sentences:
            topics = extract_topics(sentence)  # Extract topics from the sentence
            if topics and topics.lower() != 'none':
                for topic in topics.split(","):
                    topic = topic.strip().lower()
                    topic_groups[topic].append(sentence)
        
        # Initialize dictionary for review data
        review_data = {
            "full_review": full_review,
            "atmosphere_sentence": None,
            "atmosphere_rating": None,
            "service_sentence": None,
            "service_rating": None,
            "food_sentence": None,
            "food_rating": None,
            "price_sentence": None,
            "price_rating": None,
        }

        # Process each topic and its grouped sentences
        for topic, sentences_for_topic in topic_groups.items():
            sentiment_rating = analyze_sentiment(sentences_for_topic)  # Perform sentiment analysis
            if sentiment_rating:
                # Assign the first sentence for the topic and the sentiment rating
                review_data[f"{topic}_sentence"] = " ".join(sentences_for_topic)
                review_data[f"{topic}_rating"] = sentiment_rating
        
        # Append review data to results
        results.append(review_data)
    
    # Convert the list of results into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df


# Process the reviews and create the detailed DataFrame
detailed_reviews_df = process_reviews(reviews_subset)

# Display the resulting DataFrame
print(detailed_reviews_df)


### Performance if we handle all None as zeros

# Replace None with 0 in predicted ratings
# predicted_food_ratings = [0 if x is None else x for x in detailed_reviews_df["food_rating"]]
# predicted_service_ratings = [0 if x is None else x for x in detailed_reviews_df["service_rating"]]
# predicted_atmosphere_ratings = [0 if x is None else x for x in detailed_reviews_df["atmosphere_rating"]]

# Compute metrics
food_accuracy_with_zeros, food_mae_with_zeros = compute_metrics(reviews_subset["dining_stars_food"], detailed_reviews_df["food_rating"])
service_accuracy_with_zeros, service_mae_with_zeros = compute_metrics(reviews_subset["dining_stars_service"], detailed_reviews_df["service_rating"])
atmosphere_accuracy_with_zeros, atmosphere_mae_with_zeros = compute_metrics(reviews_subset["dining_stars_atmosphere"], detailed_reviews_df["atmosphere_rating"])

# Print results
print(f"Food rating (with 0s) - Accuracy: {food_accuracy_with_zeros:.2f}%, MAE: {food_mae_with_zeros:.2f}")
print(f"Service rating (with 0s) - Accuracy: {service_accuracy_with_zeros:.2f}%, MAE: {service_mae_with_zeros:.2f}")
print(f"Atmosphere rating (with 0s) - Accuracy: {atmosphere_accuracy_with_zeros:.2f}%, MAE: {atmosphere_mae_with_zeros:.2f}")


                                           full_review  \
0    Also ich hab da gerade gefrühstückt!!!\nIch fü...   
1    Ein schönes Café für den kurzen Zwischenstopp....   
2    Zum Start von unserer Urlaubsreise haben wir u...   
3    Tolles Kaffe gute Backwaren ,l.die Mädels schm...   
4    Die Erfahrung mit der Bäckerei Schmidt in Duss...   
..                                                 ...   
295                                             Super!   
296  +\n• Sehr schöne Atmosphäre / Toller Biergarte...   
297  Wir hatten einen wunderbaren Besuch in diesem ...   
298  Das Essen war reichlich und sehr lecker, das P...   
299  War sehr lecker, schön angerichtet und der Ser...   

                                   atmosphere_sentence  atmosphere_rating  \
0                                                 None                NaN   
1    Meine Auffassung war, dass es hier sehr ruhig ...                5.0   
2                                                 None                Na

In [42]:
# dataset von juliana, nur die behalten die ein subrating haben und MSE und MAE berechnen
# summary besser prompten 

In [44]:
def extract_topics(sentence):
    prompt = (
        "Classify the following sentence into one or more of these categories: "
        "'food', 'service', 'atmosphere', 'price'.\n\n"
        "### Definitions:\n"
        "- 'Food': Any mention of the taste, quality, presentation, or variety of dishes.\n"
        "- 'Service': Any mention of staff behavior, speed of service, attentiveness, or professionalism.\n"
        "- 'Atmosphere': Any mention of the restaurant's ambiance, cleanliness, noise levels, or decor.\n"
        "- 'Price': Any mention of cost, value for money, or pricing.\n\n"
        "If no category applies, respond 'none'.\n\n"
        "### Example Responses:\n"
        "- Sentence: 'The pasta was delicious and well-cooked.'\n"
        "  Categories: food\n"
        "- Sentence: 'The waiter was very polite and attentive.'\n"
        "  Categories: service\n"
        "- Sentence: 'The restaurant had a cozy and welcoming atmosphere.'\n"
        "  Categories: atmosphere\n"
        "- Sentence: 'I found the prices to be a bit high for the portion size.'\n"
        "  Categories: price\n"
        "- Sentence: 'It was raining heavily outside.'\n"
        "  Categories: none\n\n"
        "Sentence: {sentence}\n\n"
        "Categories:"
    )

    try: 
        # Call the OpenAI API with the refined prompt
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[ 
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt.format(sentence=sentence)}
            ],
            max_tokens=150
        )
        
        generated_text = response.choices[0].message['content'].strip()
        return generated_text

    except Exception as e:
        print(f"Error in extracting topics: {e}")
        return None


def process_reviews(reviews_subset):
    # Create a list to store results
    results = []

    for index, row in reviews_subset.iterrows():
        full_review = row['review_text']  # Full review text
        
        # Check if the review is missing or invalid
        if pd.isna(full_review) or not isinstance(full_review, str):
            print(f"Skipping review {index} due to invalid review text.")
            continue
        
        sentences = re.split(r'(?<=[.!?])\s+', full_review.strip())  # Split into sentences
        
        # Dictionary to store sentences by topic
        topic_groups = defaultdict(list)
        
        for sentence in sentences:
            topics = extract_topics(sentence)  # Extract topics from the sentence
            if topics and topics.lower() != 'none':
                for topic in topics.split(","):
                    topic = topic.strip().lower()
                    topic_groups[topic].append(sentence)
        
        # Initialize dictionary for review data
        review_data = {
            "full_review": full_review,
            "atmosphere_sentence": None,
            "atmosphere_rating": None,
            "service_sentence": None,
            "service_rating": None,
            "food_sentence": None,
            "food_rating": None,
            "price_sentence": None,
            "price_rating": None,
        }

        # Process each topic and its grouped sentences
        for topic, sentences_for_topic in topic_groups.items():
            sentiment_rating = analyze_sentiment(sentences_for_topic)  # Perform sentiment analysis
            if sentiment_rating:
                # Assign the first sentence for the topic and the sentiment rating
                review_data[f"{topic}_sentence"] = " ".join(sentences_for_topic)
                review_data[f"{topic}_rating"] = sentiment_rating
        
        # Append review data to results
        results.append(review_data)
    
    # Convert the list of results into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df


# Process the reviews and create the detailed DataFrame
detailed_reviews_df = process_reviews(reviews_subset)

# Display the resulting DataFrame
print(detailed_reviews_df)


### Performance if we handle all None as zeros

# Replace None with 0 in predicted ratings
# predicted_food_ratings = [0 if x is None else x for x in detailed_reviews_df["food_rating"]]
# predicted_service_ratings = [0 if x is None else x for x in detailed_reviews_df["service_rating"]]
# predicted_atmosphere_ratings = [0 if x is None else x for x in detailed_reviews_df["atmosphere_rating"]]

# Compute metrics
food_accuracy_with_zeros, food_mae_with_zeros = compute_metrics(reviews_subset["dining_stars_food"], detailed_reviews_df["food_rating"])
service_accuracy_with_zeros, service_mae_with_zeros = compute_metrics(reviews_subset["dining_stars_service"], detailed_reviews_df["service_rating"])
atmosphere_accuracy_with_zeros, atmosphere_mae_with_zeros = compute_metrics(reviews_subset["dining_stars_atmosphere"], detailed_reviews_df["atmosphere_rating"])

# Print results
print(f"Food rating (with 0s) - Accuracy: {food_accuracy_with_zeros:.2f}%, MAE: {food_mae_with_zeros:.2f}")
print(f"Service rating (with 0s) - Accuracy: {service_accuracy_with_zeros:.2f}%, MAE: {service_mae_with_zeros:.2f}")
print(f"Atmosphere rating (with 0s) - Accuracy: {atmosphere_accuracy_with_zeros:.2f}%, MAE: {atmosphere_mae_with_zeros:.2f}")


Error in analyzing sentiment: invalid literal for int() with base 10: 'I will need you to provide the sentences you would'
                                           full_review  \
0    Also ich hab da gerade gefrühstückt!!!\nIch fü...   
1    Ein schönes Café für den kurzen Zwischenstopp....   
2    Zum Start von unserer Urlaubsreise haben wir u...   
3    Tolles Kaffe gute Backwaren ,l.die Mädels schm...   
4    Die Erfahrung mit der Bäckerei Schmidt in Duss...   
..                                                 ...   
295                                             Super!   
296  +\n• Sehr schöne Atmosphäre / Toller Biergarte...   
297  Wir hatten einen wunderbaren Besuch in diesem ...   
298  Das Essen war reichlich und sehr lecker, das P...   
299  War sehr lecker, schön angerichtet und der Ser...   

                                   atmosphere_sentence  atmosphere_rating  \
0                                                 None                NaN   
1    Drinnen kann man sich

In [45]:
def extract_topics(sentence):
    prompt = (
        "For the following sentence, assign one or more categories generously from the list: "
        "'food', 'service', 'atmosphere', 'price'. A sentence can belong to multiple categories if relevant. "
        "If a category even partially applies, include it. If no category applies, respond 'none'.\n\n"
        "### Definitions:\n"
        "- 'Food': Any mention of taste, quality, presentation, portion size, variety, or availability of dishes, "
        "even if implied (e.g., 'The pizza was okay' implies food).\n"
        "- 'Service': Any mention of staff behavior, speed of service, attentiveness, professionalism, or interactions "
        "(e.g., 'We waited a while' or 'The server smiled at us').\n"
        "- 'Atmosphere': Any mention of ambiance, cleanliness, decor, noise levels, seating comfort, or overall vibe "
        "(e.g., 'The restaurant was cozy').\n"
        "- 'Price': Any mention of cost, affordability, value for money, or price-related satisfaction or dissatisfaction "
        "(e.g., 'It was expensive but worth it').\n\n"
        "### Guidelines:\n"
        "- Include all applicable categories generously. A single sentence can belong to multiple categories.\n"
        "- If a category is implied but not directly stated, still include it.\n"
        "- Be cautious about excluding categories—opt for inclusion when in doubt.\n\n"
        "### Example Responses:\n"
        "- Sentence: 'The steak was overcooked, but the waiter apologized.'\n"
        "  Categories: food, service\n"
        "- Sentence: 'The decor was lovely, and the prices were reasonable.'\n"
        "  Categories: atmosphere, price\n"
        "- Sentence: 'The food was good, but the seating was cramped.'\n"
        "  Categories: food, atmosphere\n"
        "- Sentence: 'I loved the experience!' (ambiguous but positive sentiment)\n"
        "  Categories: atmosphere\n"
        "- Sentence: 'Nothing to complain about.' (too vague for specific categories)\n"
        "  Categories: none\n\n"
        "Sentence: {sentence}\n\n"
        "Categories:"
    )

    try: 
        # Call the OpenAI API with the refined prompt
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[ 
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt.format(sentence=sentence)}
            ],
            max_tokens=150
        )
        
        generated_text = response.choices[0].message['content'].strip()
        return generated_text

    except Exception as e:
        print(f"Error in extracting topics: {e}")
        return None


def process_reviews(reviews_subset):
    # Create a list to store results
    results = []

    for index, row in reviews_subset.iterrows():
        full_review = row['review_text']  # Full review text
        
        # Check if the review is missing or invalid
        if pd.isna(full_review) or not isinstance(full_review, str):
            print(f"Skipping review {index} due to invalid review text.")
            continue
        
        sentences = re.split(r'(?<=[.!?])\s+', full_review.strip())  # Split into sentences
        
        # Dictionary to store sentences by topic
        topic_groups = defaultdict(list)
        
        for sentence in sentences:
            topics = extract_topics(sentence)  # Extract topics from the sentence
            if topics and topics.lower() != 'none':
                for topic in topics.split(","):
                    topic = topic.strip().lower()
                    topic_groups[topic].append(sentence)
        
        # Initialize dictionary for review data
        review_data = {
            "full_review": full_review,
            "atmosphere_sentence": None,
            "atmosphere_rating": None,
            "service_sentence": None,
            "service_rating": None,
            "food_sentence": None,
            "food_rating": None,
            "price_sentence": None,
            "price_rating": None,
        }

        # Process each topic and its grouped sentences
        for topic, sentences_for_topic in topic_groups.items():
            sentiment_rating = analyze_sentiment(sentences_for_topic)  # Perform sentiment analysis
            if sentiment_rating:
                # Assign the first sentence for the topic and the sentiment rating
                review_data[f"{topic}_sentence"] = " ".join(sentences_for_topic)
                review_data[f"{topic}_rating"] = sentiment_rating
        
        # Append review data to results
        results.append(review_data)
    
    # Convert the list of results into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df


# Process the reviews and create the detailed DataFrame
detailed_reviews_df = process_reviews(reviews_subset)

# Display the resulting DataFrame
print(detailed_reviews_df)


### Performance if we handle all None as zeros

# Replace None with 0 in predicted ratings
# predicted_food_ratings = [0 if x is None else x for x in detailed_reviews_df["food_rating"]]
# predicted_service_ratings = [0 if x is None else x for x in detailed_reviews_df["service_rating"]]
# predicted_atmosphere_ratings = [0 if x is None else x for x in detailed_reviews_df["atmosphere_rating"]]

# Compute metrics
food_accuracy_with_zeros, food_mae_with_zeros = compute_metrics(reviews_subset["dining_stars_food"], detailed_reviews_df["food_rating"])
service_accuracy_with_zeros, service_mae_with_zeros = compute_metrics(reviews_subset["dining_stars_service"], detailed_reviews_df["service_rating"])
atmosphere_accuracy_with_zeros, atmosphere_mae_with_zeros = compute_metrics(reviews_subset["dining_stars_atmosphere"], detailed_reviews_df["atmosphere_rating"])

# Print results
print(f"Food rating (with 0s) - Accuracy: {food_accuracy_with_zeros:.2f}%, MAE: {food_mae_with_zeros:.2f}")
print(f"Service rating (with 0s) - Accuracy: {service_accuracy_with_zeros:.2f}%, MAE: {service_mae_with_zeros:.2f}")
print(f"Atmosphere rating (with 0s) - Accuracy: {atmosphere_accuracy_with_zeros:.2f}%, MAE: {atmosphere_mae_with_zeros:.2f}")


Error in analyzing sentiment: invalid literal for int() with base 10: "I'm sorry, but I need more context or a"
                                           full_review  \
0    Also ich hab da gerade gefrühstückt!!!\nIch fü...   
1    Ein schönes Café für den kurzen Zwischenstopp....   
2    Zum Start von unserer Urlaubsreise haben wir u...   
3    Tolles Kaffe gute Backwaren ,l.die Mädels schm...   
4    Die Erfahrung mit der Bäckerei Schmidt in Duss...   
..                                                 ...   
295                                             Super!   
296  +\n• Sehr schöne Atmosphäre / Toller Biergarte...   
297  Wir hatten einen wunderbaren Besuch in diesem ...   
298  Das Essen war reichlich und sehr lecker, das P...   
299  War sehr lecker, schön angerichtet und der Ser...   

                                   atmosphere_sentence  atmosphere_rating  \
0                                                 None                NaN   
1    Meine Auffassung war, dass es hi